In [ ]:
"""
Title: Cross-Regional Adaptation Studies in Earthquake Magnitude Estimation

This notebook investigates transfer learning effectiveness for adapting magnitude 
estimation models across geological regions. Key components:
1. Performance evaluation on out-of-distribution regional data
2. Transfer learning implementation 
3. Comparative analysis with baseline performance

Dependencies:
- torch, numpy, json
- matplotlib (for visualization)
- h5py (for seismic data loading)
"""

# Part 1: Setup and Imports

In [ ]:
#------------------------------------------------------------------------------
# Part 1: Setup and Imports
#------------------------------------------------------------------------------

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import json
import os
from torchinfo import summary
import time
from tqdm import tqdm

# Configure environment
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Part 2: Dataset Class

In [ ]:
#------------------------------------------------------------------------------
# Part 2: Dataset Class
#------------------------------------------------------------------------------

class EarthquakeDataset(Dataset):
    """Dataset class handling waveforms, labels, and log stream max values."""
    def __init__(self, data, labels, log_stream_max):
        self.data = data
        self.labels = labels
        self.log_stream_max = log_stream_max

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.data[idx], self.log_stream_max[idx], self.labels[idx]

# Part 3: Model Components

In [ ]:
#------------------------------------------------------------------------------
# Part 3: Model Components
#------------------------------------------------------------------------------

class ResidualBlock(nn.Module):
    """Residual block with batch normalization."""
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv1d(in_channels, out_channels, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.conv2 = nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU()
        self.downsample = nn.Conv1d(in_channels, out_channels, kernel_size=1) if in_channels != out_channels else None

    def forward(self, x):
        residual = x
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

class EarthquakeModel(nn.Module):
    """Enhanced architecture with residual connections and log stream max."""
    def __init__(self):
        super(EarthquakeModel, self).__init__()
        self.conv1 = nn.Conv1d(3, 64, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.res3 = ResidualBlock(128, 128)
        self.res4 = ResidualBlock(128, 64)
        self.res5 = ResidualBlock(64, 32)
        self.maxpool = nn.MaxPool1d(2, padding=1)
        self.dropout = nn.Dropout(0.2)
        self.lstm1 = nn.LSTM(32, 100, batch_first=True, bidirectional=True)
        self.lstm2 = nn.LSTM(200, 200, batch_first=True, bidirectional=True)
        self.fc = nn.Linear(400 + 3, 2)

    def forward(self, x, log_stream_max):
        x = x.transpose(1, 2)
        x = self.conv1(x)
        x = self.dropout(x)
        x = self.maxpool(x)
        x = self.conv2(x)
        x = self.dropout(x)
        x = self.maxpool(x)
        x = self.res3(x)
        x = self.dropout(x)
        x = self.maxpool(x)
        x = self.res4(x)
        x = self.dropout(x)
        x = self.maxpool(x)
        x = self.res5(x)
        x = self.dropout(x)
        x = self.maxpool(x)
        x = x.transpose(1, 2)
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x = x[:, -1, :]
        combined = torch.cat((x, log_stream_max), dim=1)
        x = self.fc(combined)
        return x

# Part 4: Training Components

In [ ]:
#------------------------------------------------------------------------------
# Part 4: Training Components
#------------------------------------------------------------------------------

class EarlyStopping:
    """Early stopping with model checkpointing."""
    def __init__(self, patience=7, verbose=False, delta=0, path='best_model.pth'):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = float('inf')
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f})')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

def custom_loss(y_pred, y_true):
    """Custom loss combining prediction error and uncertainty."""
    y_hat = y_pred[:, 0]
    s = y_pred[:, 1]
    return torch.mean(0.5 * torch.exp(-s) * (y_true - y_hat)**2 + 0.5 * s)

# Part 5: Transfer Learning Functions

In [ ]:
#------------------------------------------------------------------------------
# Part 5: Transfer Learning Functions
#------------------------------------------------------------------------------

def transfer_learning(model, train_loader, val_loader, num_epochs=300, patience=50, 
                     model_path='transfer_learning_model.pth'):
    """Implement transfer learning by freezing convolutional layers."""
    model.to(device)

    # Freeze convolutional layers
    for param in model.conv1.parameters():
        param.requires_grad = False
    for param in model.conv2.parameters():
        param.requires_grad = False
    for param in model.res3.parameters():
        param.requires_grad = False
    for param in model.res4.parameters():
        param.requires_grad = False
    for param in model.res5.parameters():
        param.requires_grad = False

    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='min', factor=0.5, patience=15, verbose=True, min_lr=0.5e-6)
    early_stopping = EarlyStopping(patience=patience, verbose=True, path=model_path)

    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for data, log_stream_max, target in train_loader:
            data = data.to(device)
            log_stream_max = log_stream_max.to(device)
            target = target.to(device)
            
            optimizer.zero_grad()
            outputs = model(data, log_stream_max)
            loss = custom_loss(outputs, target)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            running_loss += loss.item()

        val_loss = 0.0
        model.eval()
        with torch.no_grad():
            for data, log_stream_max, target in val_loader:
                data = data.to(device)
                log_stream_max = log_stream_max.to(device)
                target = target.to(device)
                outputs = model(data, log_stream_max)
                loss = custom_loss(outputs, target)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        running_loss /= len(train_loader)

        scheduler.step(val_loss)
        early_stopping(val_loss, model)

        print(f'Epoch {epoch+1}, Loss: {running_loss:.4f}, Validation Loss: {val_loss:.4f}')
        print(f'Learning Rate: {optimizer.param_groups[0]["lr"]:.6f}')

        train_losses.append(running_loss)
        val_losses.append(val_loss)

        if early_stopping.early_stop:
            print(f'Early stopping triggered at epoch {epoch+1}')
            break

    return train_losses, val_losses

# Part 6: Evaluation Functions

In [ ]:
#------------------------------------------------------------------------------
# Part 6: Evaluation Functions
#------------------------------------------------------------------------------

def estimate_uncertainty(model, data_loader, num_samples=50):
    """Estimate model uncertainties using Monte Carlo dropout."""
    model.to(device)
    model.eval()

    for m in model.modules():
        if isinstance(m, nn.Dropout):
            m.train()

    predictions = []
    log_variances = []

    with torch.no_grad():
        for _ in range(num_samples):
            batch_predictions = []
            batch_log_variances = []
            for data, log_stream_max, _ in data_loader:
                data = data.to(device)
                log_stream_max = log_stream_max.to(device)
                output = model(data, log_stream_max)
                batch_predictions.append(output[:, 0].cpu().numpy())
                batch_log_variances.append(output[:, 1].cpu().numpy())
            predictions.append(np.concatenate(batch_predictions))
            log_variances.append(np.concatenate(batch_log_variances))

    predictions = np.array(predictions)
    log_variances = np.array(log_variances)

    mean_prediction = np.mean(predictions, axis=0)
    yhat_squared_mean = np.mean(np.square(predictions), axis=0)

    sigma_squared = np.power(10, log_variances)
    aleatoric_uncertainty = np.mean(sigma_squared, axis=0)

    epistemic_uncertainty = np.std(predictions, axis=0)
    combined_uncertainty = yhat_squared_mean - np.square(mean_prediction) + aleatoric_uncertainty

    return mean_prediction, epistemic_uncertainty, aleatoric_uncertainty, combined_uncertainty

def evaluate_model(model, test_loader, model_path='transfer_learning_model.pth'):
    """Evaluate model performance and uncertainties."""
    model.load_state_dict(torch.load(model_path))
    mean_pred, epistemic_unc, aleatoric_unc, combined_unc = estimate_uncertainty(model, test_loader)

    true_values = []
    for _, _, target in test_loader:
        true_values.append(target.numpy())
    true_values = np.concatenate(true_values)

    mae = np.mean(np.abs(mean_pred - true_values))

    return mae, mean_pred, true_values, epistemic_unc, aleatoric_unc, combined_unc

# Part 7: Visualization Functions

In [ ]:
#------------------------------------------------------------------------------
# Part 7: Visualization Functions
#------------------------------------------------------------------------------

def plot_uncertainties(true_values, predictions, epistemic_unc, aleatoric_unc, 
                      combined_unc, seed):
    """Plot uncertainty distributions and prediction results."""
    aleatoric_threshold = np.percentile(aleatoric_unc, 99.995)
    epistemic_threshold = np.percentile(epistemic_unc, 99.995)
    combined_threshold = np.percentile(combined_unc, 99.995)

    # Aleatoric Uncertainty
    plt.figure(figsize=(10, 8))
    mask = aleatoric_unc < aleatoric_threshold
    plt.errorbar(predictions[mask], aleatoric_unc[mask], 
                xerr=aleatoric_unc[mask], fmt='o', alpha=0.4, 
                ecolor='g', capthick=2)
    plt.plot(true_values[mask], aleatoric_unc[mask], 'ro', alpha=0.4)
    plt.xlabel('Predicted Magnitude', fontsize=14, fontweight='bold')
    plt.ylabel('Aleatoric Uncertainty', fontsize=14, fontweight='bold')
    plt.ylim(0, aleatoric_threshold)
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.tight_layout()
    plt.savefig(f'aleatoric_uncertainty_seed_{seed}.png')
    plt.close()

    # Epistemic Uncertainty
    plt.figure(figsize=(10, 8))
    mask = epistemic_unc < epistemic_threshold
    plt.errorbar(predictions[mask], epistemic_unc[mask], 
                xerr=epistemic_unc[mask], fmt='o', alpha=0.4, 
                ecolor='g', capthick=2)
    plt.plot(true_values[mask], epistemic_unc[mask], 'ro', alpha=0.4)
    plt.xlabel('Predicted Magnitude', fontsize=14, fontweight='bold')
    plt.ylabel('Epistemic Uncertainty', fontsize=14, fontweight='bold')
    plt.ylim(0, epistemic_threshold)
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.tight_layout()
    plt.savefig(f'epistemic_uncertainty_seed_{seed}.png')
    plt.close()

    # Combined Uncertainty
    plt.figure(figsize=(10, 8))
    mask = combined_unc < combined_threshold
    plt.errorbar(predictions[mask], combined_unc[mask], 
                xerr=combined_unc[mask], fmt='o', alpha=0.4, 
                ecolor='g', capthick=2)
    plt.plot(true_values[mask], combined_unc[mask], 'ro', alpha=0.4)
    plt.xlabel('Predicted Magnitude', fontsize=14, fontweight='bold')
    plt.ylabel('Combined Uncertainty', fontsize=14, fontweight='bold')
    plt.ylim(0, combined_threshold)
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.tight_layout()
    plt.savefig(f'combined_uncertainty_seed_{seed}.png')
    plt.close()

    # Predicted vs True
    plt.figure(figsize=(10, 8))
    plt.scatter(true_values, predictions, alpha=0.4, 
               facecolors='none', edgecolors='r')
    plt.plot([true_values.min(), true_values.max()], 
             [true_values.min(), true_values.max()], 
             'k--', alpha=0.4, lw=2)
    plt.xlabel('True Magnitude', fontsize=14, fontweight='bold')
    plt.ylabel('Predicted Magnitude', fontsize=14, fontweight='bold')
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.tight_layout()
    plt.savefig(f'predicted_vs_true_seed_{seed}.png')
    plt.close()

# Part 8: Main Execution

In [ ]:
#------------------------------------------------------------------------------
# Part 8: Main Execution
#------------------------------------------------------------------------------

if __name__ == "__main__":
    # Start timing
    start_time = time.time()

    # Set random seed
    seed = 1024
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

    # Define file paths
    train_val_data_file = "TL_MO_SCEDC_pre_processed_data_train_val.npy"
    train_val_labels_file = "TL_MO_SCEDC_pre_processed_labels_train_val.npy"
    train_val_log_stream_max_file = "TL_MO_SCEDC_log_stream_max_train_val.npy"
    
    test_data_file = "TL_MO_SCEDC_pre_processed_data_test.npy"
    test_labels_file = "TL_MO_SCEDC_pre_processed_labels_test.npy"
    test_log_stream_max_file = "TL_MO_SCEDC_log_stream_max_test.npy"

    # Verify files exist
    for file_path in [train_val_data_file, train_val_labels_file, 
                     train_val_log_stream_max_file, test_data_file, 
                     test_labels_file, test_log_stream_max_file]:
        assert os.path.isfile(file_path), f"File not found at {file_path}"

    # Load train/validation data
    print("\nLoading train/validation data...")
    train_val_data = np.load(train_val_data_file)
    train_val_data = train_val_data.transpose(0, 2, 1)
    train_val_data = torch.tensor(train_val_data, dtype=torch.float32)

    train_val_labels = torch.tensor(np.load(train_val_labels_file), 
                                  dtype=torch.float32)
    train_val_log_stream_max = torch.tensor(np.load(train_val_log_stream_max_file), 
                                          dtype=torch.float32)

    # Load test data
    print("\nLoading test data...")
    test_data = np.load(test_data_file)
    test_data = test_data.transpose(0, 2, 1)
    test_data = torch.tensor(test_data, dtype=torch.float32)

    test_labels = torch.tensor(np.load(test_labels_file), dtype=torch.float32)
    test_log_stream_max = torch.tensor(np.load(test_log_stream_max_file), 
                                     dtype=torch.float32)

    # Print data shapes
    print(f"\nData Shapes:")
    print(f"Train/Val Data: {train_val_data.shape}")
    print(f"Train/Val Labels: {train_val_labels.shape}")
    print(f"Train/Val Log Stream Max: {train_val_log_stream_max.shape}")
    print(f"Test Data: {test_data.shape}")
    print(f"Test Labels: {test_labels.shape}")
    print(f"Test Log Stream Max: {test_log_stream_max.shape}")

    # Split train/validation data
    total_samples = len(train_val_data)
    train_size = int(8/9 * total_samples)
    val_size = total_samples - train_size

    print(f"\nSplitting train/val data:")
    print(f"Total samples: {total_samples}")
    print(f"Training samples: {train_size}")
    print(f"Validation samples: {val_size}")

    train_data = train_val_data[:train_size]
    val_data = train_val_data[train_size:]
    train_labels = train_val_labels[:train_size]
    val_labels = train_val_labels[train_size:]
    train_log_stream_max = train_val_log_stream_max[:train_size]
    val_log_stream_max = train_val_log_stream_max[train_size:]

    # Create datasets
    train_dataset = EarthquakeDataset(train_data, train_labels, train_log_stream_max)
    val_dataset = EarthquakeDataset(val_data, val_labels, val_log_stream_max)
    test_dataset = EarthquakeDataset(test_data, test_labels, test_log_stream_max)

    # Create dataloaders
    print("\nCreating dataloaders...")
    train_loader = DataLoader(train_dataset, batch_size=256, shuffle=False, num_workers=2)
    val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=2)
    test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False, num_workers=2)

    # Load pre-trained best model
    best_model_path = "best_model_seed_1024.pth"
    assert os.path.isfile(best_model_path), f"Pre-trained model not found at {best_model_path}"

    print("\nLoading pre-trained model...")
    model = EarthquakeModel()
    model.load_state_dict(torch.load(best_model_path))
    print("Model loaded successfully!")

    # Perform transfer learning
    print("\nStarting transfer learning...")
    transfer_model_path = 'transfer_learning_model.pth'
    train_losses, val_losses = transfer_learning(model, train_loader, val_loader,
                                               num_epochs=300, patience=50,
                                               model_path=transfer_model_path)

    # Evaluate transfer-learned model
    print("\nEvaluating model...")
    mae, mean_pred, true_values, epistemic_unc, aleatoric_unc, combined_unc = \
        evaluate_model(model, test_loader, model_path=transfer_model_path)

    # Print results
    print(f'\nTransfer Learning Results:')
    print(f'Mean Absolute Error (MAE): {mae:.4f}')
    print(f'Mean Aleatoric Uncertainty: {np.mean(aleatoric_unc):.4f}')
    print(f'Mean Epistemic Uncertainty: {np.mean(epistemic_unc):.4f}')
    print(f'Mean Combined Uncertainty: {np.mean(combined_unc):.4f}')

    # Plot uncertainties
    print("\nGenerating plots...")
    plot_uncertainties(true_values, mean_pred, epistemic_unc, 
                      aleatoric_unc, combined_unc, seed)

    # Plot loss curves
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.xlabel('Epochs', fontsize=14, fontweight='bold')
    plt.ylabel('Loss', fontsize=14, fontweight='bold')
    plt.legend(fontsize=14)
    plt.grid(True)
    plt.tick_params(axis='both', which='major', labelsize=14)
    plt.tight_layout()
    plt.savefig('transfer_learning_loss_curves.png')
    plt.close()

    # Save results
    results = {
        "MAE": float(mae),
        "Mean_Aleatoric_Uncertainty": float(np.mean(aleatoric_unc)),
        "Mean_Epistemic_Uncertainty": float(np.mean(epistemic_unc)),
        "Mean_Combined_Uncertainty": float(np.mean(combined_unc))
    }

    with open('transfer_learning_results.json', 'w') as f:
        json.dump(results, f, indent=4)

    # End timing
    end_time = time.time()
    print(f"\nTotal execution time: {(end_time - start_time)/60:.2f} minutes")